# An introduction to `relatio` 
**Runtime $\sim$ 1h**

Original paper: ["Text Semantics Capture Political and Economic Narratives"](https://arxiv.org/abs/2108.01720)

----------------------------

This is a short demo of the package `relatio`.  It takes as input a text corpus and outputs a list of narrative statements. The pipeline is unsupervised: the user does not need to specify narratives beforehand. Narrative statements are defined as tuples of semantic roles with a (agent, verb, patient, attribute) structure. 

Here, we present the main wrapper functions to quickly obtain narrative statements from a corpus.

----------------------------

We provide datasets that have already been split into sentences and annotated by our team.

The datasets are provided in three different formats:
 1. `raw` (unprocessed)
 2. `split_sentences` (as a list of sentences)
 3. `srl` (as a list of annotated sentences by the semantic role labeler)

In this tutorial, we work with the Trump Tweet Archive corpus.

----------------------------

In [1]:
# Catch warnings for an easy ride
from relatio._logging import FileLogger
logger = FileLogger(level = 'WARNING')

2022-03-02 18:20:43.010871: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-02 18:20:43.010890: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Browse list of available datasets
from relatio.datasets import list_datasets
print(list_datasets())

# Load an available dataset
from relatio.datasets import load_trump_data
df = load_trump_data("raw")


    List of available datasets:

    Trump Tweet Archive
    - function call: load_trump_data()
    - format: 'raw', 'split_sentences', 'srl_res'
    - allennlp version: 0.9
    - srl model: srl-model-2018.05.25.tar.gz
    


## Step 1: Split into sentences

----------------------------

For any new corpus, the first thing you will want to do is to split the corpus into sentences.

We do this on the first 100 tweets. 

The output is two lists: one with an index for the document and one with the resulting split sentences.

----------------------------


In [3]:
from relatio.preprocessing import *

p = Preprocessor(
    spacy_model = "en_core_web_md",
    remove_punctuation = True,
    remove_digits = True,
    lowercase = True,
    lemmatize = True,
    stop_words = [],
    n_process = -1,
    batch_size = 100
)

split_sentences = p.split_into_sentences(
    df.iloc[0:100], output_path='sentences.json', progress_bar=True
)

from relatio.utils import load_sentences
doc_index, sentences = load_sentences('sentences.json')

Splitting into sentences...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 358.83it/s]


## Step 2: Annotate semantic roles

----------------------------

Once the corpus is split into sentences. You can feed it to the semantic role labeler.

The output is a list of json objects which contain the semantic role annotations for each sentence in the corpus.

----------------------------


In [4]:
from relatio.semantic_role_labeling import *

SRL = SRL(
    path = "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",
    batch_size = 10,
    cuda_device = -1
)

srl_res = SRL(split_sentences[1], progress_bar=True)

Running SRL...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:05<00:00,  3.93it/s]


In [5]:
srl_res[0]

{'verbs': [{'verb': 'have',
   'description': 'Republicans and Democrats [V: have] both created our economic problems .',
   'tags': ['O', 'O', 'O', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O']},
  {'verb': 'created',
   'description': '[ARG0: Republicans and Democrats] have both [V: created] [ARG1: our economic problems] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'I-ARG0',
    'O',
    'O',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']}],
 'words': ['Republicans',
  'and',
  'Democrats',
  'have',
  'both',
  'created',
  'our',
  'economic',
  'problems',
  '.']}

NB: This step is faster with a GPU. The argument cuda_device allows users to use their GPUs:

```
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())

SRL = SRL(
    path = "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",
    batch_size = 10,
    cuda_device = 0
)

srl_res = SRL(split_sentences[1], progress_bar=True)
```

In [6]:
# To save us some time, we download the results from the datasets module.
split_sentences = load_trump_data("split_sentences")
srl_res = load_trump_data("srl_res")

## Step 3: Pre-process semantic roles

----------------------------


----------------------------

In [7]:
roles, sentence_index = p.extract_roles(
    srl_res, 
    used_roles = ["ARG0","B-V","B-ARGM-NEG","B-ARGM-MOD","ARG1","ARG2"],
    progress_bar = True
)

for d in roles[0:5]: print(d)

Extracting semantic roles...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68616/68616 [00:01<00:00, 38474.10it/s]

{'B-V': 'have'}
{'ARG0': 'Republicans and Democrats', 'ARG1': 'our economic problems', 'B-V': 'created'}
{'ARG1': 'I', 'ARG2': 'thrilled to be back in the Great city of Charlotte , North Carolina with thousands of hardworking American Patriots who love our Country , cherish our values , respect our laws , and always put AMERICA FIRST', 'B-V': 'was'}
{'ARG1': 'I', 'ARG2': 'to be back in the Great city of Charlotte , North Carolina with thousands of hardworking American Patriots who love our Country , cherish our values , respect our laws , and always put AMERICA FIRST', 'B-V': 'thrilled'}
{'ARG1': 'I', 'ARG2': 'back in the Great city of Charlotte , North Carolina', 'B-V': 'be'}


In [8]:
postproc_roles = p.process_roles(roles, 
                                 dict_of_pos_tags_to_keep = {
                                     "ARG0": ['NOUN', 'PROPN'],
                                     "B-V": ['VERB'],
                                     "ARG1": ['NOUN', 'PROPN'],
                                     "ARG2": ['NOUN', 'PROPN']
                                 }, 
                                 progress_bar = True,
                                 output_path = 'postproc_roles.json')

from relatio.utils import load_roles
postproc_roles = load_roles('postproc_roles.json')

for d in postproc_roles[0:5]: print(d)

Cleaning roles ARG0...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49281/49281 [00:20<00:00, 2406.24it/s]


Cleaning roles B-V...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 135562/135562 [00:55<00:00, 2445.26it/s]


Cleaning roles B-ARGM-MOD...


...

Cleaning roles ARG1...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88730/88730 [00:41<00:00, 2119.53it/s]


Cleaning roles ARG2...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32132/32132 [00:15<00:00, 2124.04it/s]


{'B-V': 'have'}
{'ARG0': 'republicans democrats', 'B-V': 'create', 'ARG1': 'problem'}
{'B-V': '', 'ARG1': '', 'ARG2': 'city charlotte north carolina thousand patriots country value law america'}
{'B-V': 'thrill', 'ARG1': '', 'ARG2': 'city charlotte north carolina thousand patriots country value law america'}
{'B-V': '', 'ARG1': '', 'ARG2': 'city charlotte north carolina'}


In [9]:
known_entities = p.mine_entities(
    split_sentences[1], 
    clean_entities = True, 
    progress_bar = True,
    output_path = 'entities.pkl'
)

from relatio.utils import load_entities
known_entities = load_entities('entities.pkl')

for n in known_entities.most_common(10): print(n)

Mining named entities...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68616/68616 [00:47<00:00, 1431.78it/s]


('democrats', 1180)
('obama', 1029)
('china', 926)
('u.s.', 860)
('trump', 836)
('america', 672)
('american', 542)
('barackobama', 518)
('the united states', 491)
('dems', 476)


In [10]:
top_known_entities = [e[0] for e in list(known_entities.most_common(100)) if e[0] != '']

## Step 4: Build a narrative model

----------------------------

We are now ready to build a narrative model.

----------------------------

In [11]:
from relatio.narrative_models import *
from relatio.utils import prettify
from collections import Counter

m = NarrativeModel(model_type = 'deterministic',
                   roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'B-ARGM-MOD', 'ARG1', 'ARG2'],
                   roles_with_entities = ['ARG0','ARG1','ARG2'],
                   list_of_known_entities = top_known_entities,
                   assignment_to_known_entities = 'character_matching',
                   roles_with_embeddings = [],
                   embeddings_model = None,
                   threshold = 1)    

m.train(postproc_roles)

No training required: the model is deterministic.


In [12]:
import cProfile
cProfile.run("narratives = m.predict(postproc_roles, progress_bar = True, prettify = False)")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150213/150213 [00:18<00:00, 7971.03it/s]

         73341893 function calls (72539214 primitive calls) in 19.927 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        1    0.000    0.000   19.927   19.927 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _monitor.py:94(report)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:106(remove)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:16(__init__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:20(__enter__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:26(__exit__)
        2    0.000    0.000    0.000    0.000 _weakrefset.py:52(_commit_removals)
        3    0.000    0.000    0.000    0.000 _weakrefset.py:58(__iter__)
        1    0.000    0.000    0.000    0.000 _weakrefset.py:81(add)
 802680/1    0.525    0.000    1.078    1.078 copy.py:132(deepc

In [13]:
pretty_narratives = []
for n in narratives: 
    if n.get('ARG0') not in ["", None]:
        if n.get('B-V') not in ["", None]:
            if n.get('ARG1') not in ["", None]:
                pretty_narratives.append(prettify(n))
                
pretty_narratives = Counter(pretty_narratives)
for t in pretty_narratives.most_common(10): print(t)

('administration do administration', 4)
('china take u.s.', 4)
('trump recognize russian', 3)
('democrats want country', 3)
('north carolina make republican', 3)
('fake news trace coronavirus china', 3)
('foxnews discuss barackobama', 3)
('canada look china', 3)
('foxnews discuss mittromney', 3)
('iran take iraq', 3)


In [14]:
m = NarrativeModel(model_type = 'dynamic',
                   roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'B-ARGM-MOD', 'ARG1', 'ARG2'],
                   roles_with_entities = ['ARG0','ARG1','ARG2'],
                   list_of_known_entities = top_known_entities,
                   assignment_to_known_entities = 'character_matching',
                   roles_with_embeddings = [['ARG0','ARG1','ARG2']],
                   embeddings_model = None,
                   threshold = 0.3)    

m.train(postproc_roles, progress_bar = True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150213/150213 [07:38<00:00, 327.67it/s]


In [15]:
cProfile.run("narratives = m.predict(postproc_roles, progress_bar = True, prettify = False)")

 36%|█████████████████████████████████████████████████████████▍                                                                                                    | 54588/150213 [04:19<07:33, 210.70it/s]


         61662054 function calls (58156029 primitive calls) in 260.035 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    28758    0.041    0.000    0.701    0.000 <__array_function__ internals>:2(amin)
    28759    0.025    0.000    0.144    0.000 <__array_function__ internals>:2(atleast_1d)
    86277    0.085    0.000    0.502    0.000 <__array_function__ internals>:2(concatenate)
    50677    0.050    0.000    0.166    0.000 <__array_function__ internals>:2(count_nonzero)
    28759    0.025    0.000    0.117    0.000 <__array_function__ internals>:2(dot)
    28759    0.031    0.000    0.510    0.000 <__array_function__ internals>:2(hstack)
    28759    0.034    0.000    0.412    0.000 <__array_function__ internals>:2(norm)
    28758    0.030    0.000    0.266    0.000 <__array_function__ internals>:2(where)
        3    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        1    0.

KeyboardInterrupt: 

In [ ]:
pretty_narratives = []
for n in narratives: 
    if n.get('ARG0') not in ["", None]:
        if n.get('B-V') not in ["", None]:
            if n.get('ARG1') not in ["", None]:
                pretty_narratives.append(prettify(n))
                
pretty_narratives = Counter(pretty_narratives)
for t in pretty_narratives.most_common(10): print(t)

## Step 5: Model validation and basic analysis

----------------------------


----------------------------

In [ ]:
for i,k in enumerate(sentence_index):
    
    n = narratives[i]
    r = roles[i]
    
    if n.get('ARG0') not in ["", None]:
        if n.get('B-V') not in ["", None]:
            if n.get('ARG1') not in ["", None]:
                print('Original statement:')
                print(prettify(r))
                print('\n')
                print('Underlying narrative:')
                print(prettify(n))
                print("\n")
                

## Step 6: Visualization // Plotting narrative graphs
----------------------------

A collection of narrative statements has an intuitive network structure, in which the edges are verbs and the nodes are entities.

Here, we plot Trump's narrative statements on Twitter.

----------------------------